In [120]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import ast
pd.set_option('display.max_rows', 100)

In [122]:
df = pd.read_excel('../datasets/real_estate_data.xlsx')

In [124]:
df.head()

,index,PropertyName,PropertySubName,NearbyLocations,LocationAdvantages,Link,PriceDetails,TopFacilities
0,0,Smartworld One DXP,"2, 3, 4 BHK Apartment in Sector 113, Gurgaon","['Bajghera Road', 'Palam Vihar Halt', 'DPSG Pa...","{'Bajghera Road': '800 Meter', 'Palam Vihar Ha...",https://www.99acres.com/smartworld-one-dxp-sec...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Swimming Pool Salon Restaurant Spa Cafeteria S...
1,1,M3M Crown,"3, 4 BHK Apartment in Sector 111, Gurgaon","['DPSG Palam Vihar Gurugram', 'The NorthCap Un...","{'DPSG Palam Vihar Gurugram': '1.4 Km', 'The N...",https://www.99acres.com/m3m-crown-sector-111-g...,"{'3 BHK': {'building_type': 'Apartment', 'area...",Bowling Alley Mini Theatre Manicured Garden Sw...
2,2,Adani Brahma Samsara Vilasa,"Land, 3, 4 BHK Independent Floor in Sector 63,...","['AIPL Business Club Sector 62', 'Heritage Xpe...","{'AIPL Business Club Sector 62': '2.7 Km', 'He...",https://www.99acres.com/adani-brahma-samsara-v...,{'3 BHK': {'building_type': 'Independent Floor...,Terrace Garden Gazebo Fountain Amphitheatre Pa...
3,3,Sobha City,"2, 3, 4 BHK Apartment in Sector 108, Gurgaon","['The Shikshiyan School', 'WTC Plaza', 'Luxus ...","{'The Shikshiyan School': '2.9 KM', 'WTC Plaza...",https://www.99acres.com/sobha-city-sector-108-...,"{'2 BHK': {'building_type': 'Apartment', 'area...",Swimming Pool Volley Ball Court Aerobics Centr...
4,4,Signature Global City 93,"2, 3 BHK Independent Floor in Sector 93 Gurgaon","['Pranavananda Int. School', 'DLF Site central...","{'Pranavananda Int. School': '450 m', 'DLF Sit...",https://www.99acres.com/signature-global-city-...,{'2 BHK': {'building_type': 'Independent Floor...,Mini Theatre Doctor on Call Concierge Service ...


In [126]:
# df.drop(22, inplace=True)
df.reset_index(drop=True, inplace=True)

In [128]:
df.columns

Index(['index', 'PropertyName', 'PropertySubName', 'NearbyLocations',
       'LocationAdvantages', 'Link', 'PriceDetails', 'TopFacilities'],
      dtype='object')

In [130]:
df.shape

(246, 8)

In [132]:
df[['PropertyName', 'TopFacilities']]

,PropertyName,TopFacilities
0,Smartworld One DXP,Swimming Pool Salon Restaurant Spa Cafeteria S...
1,M3M Crown,Bowling Alley Mini Theatre Manicured Garden Sw...
2,Adani Brahma Samsara Vilasa,Terrace Garden Gazebo Fountain Amphitheatre Pa...
3,Sobha City,Swimming Pool Volley Ball Court Aerobics Centr...
4,Signature Global City 93,Mini Theatre Doctor on Call Concierge Service ...
...,...,...
241,DLF Princeton Estate,Swimming Pool Medical Centre Laundry Salon Gro...
242,Pyramid Urban Homes 2,Shopping Centre Community Hall 24x7 Security G...
243,Satya The Hermitage,Bus Shelter Swimming Pool Business Lounge Read...
244,BPTP Spacio,Swimming Pool Card Room Piped Gas Pool Table S...


In [134]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [136]:
# def extract_list(s):
#     extracted = re.findall(r"'(.*?)'", s)
#     return extracted

In [138]:
# df['TopFacilities'] = df['TopFacilities'].apply(extract_list)

# df['TopFacilities'] =  df['TopFacilities'].apply(lambda x: ' '.join(x))

In [140]:
tfidfvectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))
Xt1 = tfidfvectorizer.fit_transform(df['TopFacilities'])


In [142]:
sim1 = cosine_similarity(Xt1)
sim1

array([[1.        , 0.01095159, 0.        , ..., 0.01183329, 0.08656385,
        0.0110727 ],
       [0.01095159, 1.        , 0.01982121, ..., 0.11904241, 0.01555534,
        0.00963852],
       [0.        , 0.01982121, 1.        , ..., 0.07020502, 0.03820314,
        0.01962826],
       ...,
       [0.01183329, 0.11904241, 0.07020502, ..., 1.        , 0.09825738,
        0.03255851],
       [0.08656385, 0.01555534, 0.03820314, ..., 0.09825738, 1.        ,
        0.06257614],
       [0.0110727 , 0.00963852, 0.01962826, ..., 0.03255851, 0.06257614,
        1.        ]])

In [144]:
li_property_names = df['PropertyName'].tolist()

In [146]:
len(li_property_names)

246

In [148]:
li_top_facility_names = df['TopFacilities'].tolist()

In [150]:
len(li_top_facility_names)

246

In [152]:
def recommend_properties_facilities(x:str):
    ind_property = li_property_names.index(x)
    property_sims = sim1[ind_property].tolist()
    numbered_sims = list(enumerate(property_sims))
    closer = sorted(numbered_sims, key=lambda x: x[1], reverse=True)[:6]
    indices = []
    for idx, _ in closer:
        indices.append(idx)
    return df[['PropertyName']].loc[indices]

In [154]:
recommend_properties_facilities('Pioneer Park')

,PropertyName
184,Pioneer Park
201,Unitech The Residences
223,Shree Vardhman City
17,Suncity Vatsal Valley
115,Ireo City Plots
45,Signature Global City 37D Ph 2


In [156]:
import numpy as np

In [158]:
def pre(x):
    dic = ast.literal_eval(x)
    
    if '1 BHK' in dic:
        feature_dic = dic['1 BHK']
        
        custom_dic['building_type_1_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)

        # custom_dic['area_type_1_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_1_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_1_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_1_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_1_BHK'].append(np.nan)
        else:
            custom_dic['area_low_1_BHK'].append(np.nan)
            custom_dic['area_high_1_BHK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_1_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_1_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_1_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_1_BHK'].append(np.nan)
        else:
            custom_dic['price_low_1_BHK'].append(np.nan)
            custom_dic['price_high_1_BHK'].append(np.nan)
            
    else:
        custom_dic['building_type_1_BHK'].append(None)
        # custom_dic['area_type_1_BHK'].append(np.nan)
        custom_dic['area_low_1_BHK'].append(np.nan)
        custom_dic['area_high_1_BHK'].append(np.nan)
        custom_dic['price_low_1_BHK'].append(np.nan)
        custom_dic['price_high_1_BHK'].append(np.nan)
        
        
    if '2 BHK' in dic:
        feature_dic = dic['2 BHK']
        
        custom_dic['building_type_2_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)

        # custom_dic['area_type_2_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_2_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_2_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_2_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_2_BHK'].append(np.nan)
        else:
            custom_dic['area_low_2_BHK'].append(np.nan)
            custom_dic['area_high_2_BHK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_2_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_2_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_2_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_2_BHK'].append(np.nan)
        else:
            custom_dic['price_low_2_BHK'].append(np.nan)
            custom_dic['price_high_2_BHK'].append(np.nan)

    else:
        custom_dic['building_type_2_BHK'].append(None)
        # custom_dic['area_type_2_BHK'].append(np.nan)
        custom_dic['area_low_2_BHK'].append(np.nan)
        custom_dic['area_high_2_BHK'].append(np.nan)
        custom_dic['price_low_2_BHK'].append(np.nan)
        custom_dic['price_high_2_BHK'].append(np.nan)
        
    if '3 BHK' in dic:
        feature_dic = dic['3 BHK']
      
        custom_dic['building_type_3_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)

        # custom_dic['area_type_3_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)

        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_3_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_3_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_3_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_3_BHK'].append(np.nan)
        else:
            custom_dic['area_low_3_BHK'].append(np.nan)
            custom_dic['area_high_3_BHK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_3_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_3_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_3_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_3_BHK'].append(np.nan)
        else:
            custom_dic['price_low_3_BHK'].append(np.nan)
            custom_dic['price_high_3_BHK'].append(np.nan)

    else:
        custom_dic['building_type_3_BHK'].append(None)
        # custom_dic['area_type_3_BHK'].append(np.nan)
        custom_dic['area_low_3_BHK'].append(np.nan)
        custom_dic['area_high_3_BHK'].append(np.nan)
        custom_dic['price_low_3_BHK'].append(np.nan)
        custom_dic['price_high_3_BHK'].append(np.nan)
         
        
    if '4 BHK' in dic:
        feature_dic = dic['4 BHK']
        
        custom_dic['building_type_4_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)

        # custom_dic['area_type_1_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_4_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_4_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_4_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_4_BHK'].append(np.nan)
        else:
            custom_dic['area_low_4_BHK'].append(np.nan)
            custom_dic['area_high_4_BHK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_4_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_4_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_4_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_4_BHK'].append(np.nan)
        else:
            custom_dic['price_low_4_BHK'].append(np.nan)
            custom_dic['price_high_4_BHK'].append(np.nan)

    else:
        custom_dic['building_type_4_BHK'].append(None)
        # custom_dic['area_type_4_BHK'].append(np.nan)
        custom_dic['area_low_4_BHK'].append(np.nan)
        custom_dic['area_high_4_BHK'].append(np.nan)
        custom_dic['price_low_4_BHK'].append(np.nan)
        custom_dic['price_high_4_BHK'].append(np.nan)
         
        
    if '5 BHK' in dic:
        feature_dic = dic['5 BHK']
        
        custom_dic['building_type_5_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)
        
        # custom_dic['area_type_5_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_5_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_5_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_5_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_5_BHK'].append(np.nan)
        else:
            custom_dic['area_low_5_BHK'].append(np.nan)
            custom_dic['area_high_5_BHK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_5_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_5_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_5_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_5_BHK'].append(np.nan)
        else:
            custom_dic['price_low_5_BHK'].append(np.nan)
            custom_dic['price_high_5_BHK'].append(np.nan)

    else:
        custom_dic['building_type_5_BHK'].append(None)
        # custom_dic['area_type_5_BHK'].append(np.nan)
        custom_dic['area_low_5_BHK'].append(np.nan)
        custom_dic['area_high_5_BHK'].append(np.nan)
        custom_dic['price_low_5_BHK'].append(np.nan)
        custom_dic['price_high_5_BHK'].append(np.nan)
         
        
    if '6 BHK' in dic:
        feature_dic = dic['6 BHK']
      
        custom_dic['building_type_6_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)

        # custom_dic['area_type_6_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_6_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_6_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_6_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_6_BHK'].append(np.nan)
        else:
            custom_dic['area_low_6_BHK'].append(np.nan)
            custom_dic['area_high_6_BHK'].append(np.nan)
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_6_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_6_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_6_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_6_BHK'].append(np.nan)
        else:
            custom_dic['price_low_6_BHK'].append(np.nan)
            custom_dic['price_high_6_BHK'].append(np.nan)

    else:
        custom_dic['building_type_6_BHK'].append(None)
        # custom_dic['area_type_6_BHK'].append(np.nan)
        custom_dic['area_low_6_BHK'].append(np.nan)
        custom_dic['area_high_6_BHK'].append(np.nan)
        custom_dic['price_low_6_BHK'].append(np.nan)
        custom_dic['price_high_6_BHK'].append(np.nan)
         
        
    if '1 RK' in dic:
        feature_dic = dic['1 RK']
       
        custom_dic['building_type_1_RK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)

        # custom_dic['area_type_1_RK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_1_RK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_1_RK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern, area_splitted[0])
                custom_dic['area_low_1_RK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_1_RK'].append(np.nan)
        else:
            custom_dic['area_low_1_RK'].append(np.nan)
            custom_dic['area_high_1_RK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_1_RK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_1_RK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern_area, price_splitted[0])
                custom_dic['price_low_1_RK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_1_RK'].append(np.nan)
        else:
            custom_dic['price_low_1_RK'].append(np.nan)
            custom_dic['price_high_1_RK'].append(np.nan)

    else:
        custom_dic['building_type_1_RK'].append(None)
        # custom_dic['area_type_1_RK'].append(np.nan)
        custom_dic['area_low_1_RK'].append(np.nan)
        custom_dic['area_high_1_RK'].append(np.nan)
        custom_dic['price_low_1_RK'].append(np.nan)
        custom_dic['price_high_1_RK'].append(np.nan)
         
        
    if 'Land' in dic:
        feature_dic = dic['Land']
      
        custom_dic['building_type_Land_BHK'].append(feature_dic['building_type'] if 'building_type' in feature_dic else np.nan)
        
        # custom_dic['area_type_Land_BHK'].append(feature_dic['area_type'] if 'area_type' in feature_dic else np.nan)
        
        area_li = feature_dic['area'] if 'area' in feature_dic else ''
        
        if area_li:
            pattern_area = re.compile(r"\d+,?\d*")
            area_splitted = area_li.split('-')
            if len(area_splitted) > 1:
                custom_dic['area_low_Land_BHK'].append(re.findall(pattern_area, area_splitted[0].replace(',', ''))[0])
                custom_dic['area_high_Land_BHK'].append(re.findall(pattern_area, area_splitted[1].replace(',', ''))[0])
            else:
                match_area = re.findall(pattern_area, area_splitted[0].replace(',', ''))
                custom_dic['area_low_Land_BHK'].append(match_area[0] if len(match_area) != 0 else np.nan)
                custom_dic['area_high_Land_BHK'].append(np.nan)
        else:
            custom_dic['area_low_Land_BHK'].append(np.nan)
            custom_dic['area_high_Land_BHK'].append(np.nan)
        
            
        price_li = feature_dic['price-range'] if 'price-range' in feature_dic else ''
        if price_li:
            pattern = re.compile(r"[0-9]+\.?[0-9]*")
            price_splitted = price_li.split('-')
            if len(price_splitted) > 1:
                custom_dic['price_low_Land_BHK'].append(re.findall(pattern, price_splitted[0])[0])
                custom_dic['price_high_Land_BHK'].append(re.findall(pattern, price_splitted[1])[0])
            else:
                match = re.findall(pattern, price_splitted[0])
                custom_dic['price_low_Land_BHK'].append(match[0] if len(match) != 0 else np.nan)
                custom_dic['price_high_Land_BHK'].append(np.nan)
        else:
            custom_dic['price_low_Land_BHK'].append(np.nan)
            custom_dic['price_high_Land_BHK'].append(np.nan)

    else:
        custom_dic['building_type_Land_BHK'].append(None)
        # custom_dic['area_type_Land_BHK'].append(np.nan)
        custom_dic['area_low_Land_BHK'].append(np.nan)
        custom_dic['area_high_Land_BHK'].append(np.nan)
        custom_dic['price_low_Land_BHK'].append(np.nan)
        custom_dic['price_high_Land_BHK'].append(np.nan)

In [160]:
custom_dic = {}
custom_dic['building_type_1_BHK'] = []
# custom_dic['area_type_1_BHK'] = []
custom_dic['area_low_1_BHK'] = []
custom_dic['area_high_1_BHK'] = []
custom_dic['price_low_1_BHK'] = []
custom_dic['price_high_1_BHK'] = []

custom_dic['building_type_2_BHK'] = []
# custom_dic['area_type_2_BHK'] = []
custom_dic['area_low_2_BHK'] = []
custom_dic['area_high_2_BHK'] = []
custom_dic['price_low_2_BHK'] = []
custom_dic['price_high_2_BHK'] = []

custom_dic['building_type_3_BHK'] = []
# custom_dic['area_type_3_BHK'] = []
custom_dic['area_low_3_BHK'] = []
custom_dic['area_high_3_BHK'] = []
custom_dic['price_low_3_BHK'] = []
custom_dic['price_high_3_BHK'] = []

custom_dic['building_type_4_BHK'] = []
# custom_dic['area_type_4_BHK'] = []
custom_dic['area_low_4_BHK'] = []
custom_dic['area_high_4_BHK'] = []
custom_dic['price_low_4_BHK'] = []
custom_dic['price_high_4_BHK'] = []

custom_dic['building_type_5_BHK'] = []
# custom_dic['area_type_5_BHK'] = []
custom_dic['area_low_5_BHK'] = []
custom_dic['area_high_5_BHK'] = []
custom_dic['price_low_5_BHK'] = []
custom_dic['price_high_5_BHK'] = []

custom_dic['building_type_6_BHK'] = []
# custom_dic['area_type_6_BHK'] = []
custom_dic['area_low_6_BHK'] = []
custom_dic['area_high_6_BHK'] = []
custom_dic['price_low_6_BHK'] = []
custom_dic['price_high_6_BHK'] = []

custom_dic['building_type_1_RK'] = []
# custom_dic['area_type_1_RK'] = []
custom_dic['area_low_1_RK'] = []
custom_dic['area_high_1_RK'] = []
custom_dic['price_low_1_RK'] = []
custom_dic['price_high_1_RK'] = []

custom_dic['building_type_Land_BHK'] = []
# custom_dic['area_type_Land_BHK'] = []
custom_dic['area_low_Land_BHK'] = []
custom_dic['area_high_Land_BHK'] = []
custom_dic['price_low_Land_BHK'] = []
custom_dic['price_high_Land_BHK'] = []

In [162]:
for i in df['PriceDetails']:
    pre(i)

In [164]:
new_df = pd.DataFrame(custom_dic)

In [166]:
li = ['area', 'price']
for i in new_df.columns:
    splitted = i.split("_")
    if splitted[0] in li:
        new_df[f'{i}'] = new_df[f'{i}'].astype('float')

In [168]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   building_type_1_BHK     18 non-null     object 
 1   area_low_1_BHK          18 non-null     float64
 2   area_high_1_BHK         9 non-null      float64
 3   price_low_1_BHK         8 non-null      float64
 4   price_high_1_BHK        5 non-null      float64
 5   building_type_2_BHK     121 non-null    object 
 6   area_low_2_BHK          121 non-null    float64
 7   area_high_2_BHK         73 non-null     float64
 8   price_low_2_BHK         95 non-null     float64
 9   price_high_2_BHK        72 non-null     float64
 10  building_type_3_BHK     186 non-null    object 
 11  area_low_3_BHK          186 non-null    float64
 12  area_high_3_BHK         122 non-null    float64
 13  price_low_3_BHK         163 non-null    float64
 14  price_high_3_BHK        133 non-null    fl

In [170]:
pd.set_option('display.max_columns', None)

new_df

,building_type_1_BHK,area_low_1_BHK,area_high_1_BHK,price_low_1_BHK,price_high_1_BHK,building_type_2_BHK,area_low_2_BHK,area_high_2_BHK,price_low_2_BHK,price_high_2_BHK,building_type_3_BHK,area_low_3_BHK,area_high_3_BHK,price_low_3_BHK,price_high_3_BHK,building_type_4_BHK,area_low_4_BHK,area_high_4_BHK,price_low_4_BHK,price_high_4_BHK,building_type_5_BHK,area_low_5_BHK,area_high_5_BHK,price_low_5_BHK,price_high_5_BHK,building_type_6_BHK,area_low_6_BHK,area_high_6_BHK,price_low_6_BHK,price_high_6_BHK,building_type_1_RK,area_low_1_RK,area_high_1_RK,price_low_1_RK,price_high_1_RK,building_type_Land_BHK,area_low_Land_BHK,area_high_Land_BHK,price_low_Land_BHK,price_high_Land_BHK
0,None,NaN,NaN,NaN,NaN,Apartment,1370.0,NaN,2.00,2.40,Apartment,1850.0,2050.0,2.25,3.59,Apartment,2600.0,NaN,3.24,4.56,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Apartment,1605.0,2170.0,2.20,3.03,Apartment,2248.0,2670.0,3.08,3.73,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,Independent Floor,1800.0,3150.0,2.43,15.75,Independent Floor,2750.0,4500.0,3.36,22.50,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,,500.0,4329.0,2.05,41.13
3,None,NaN,NaN,NaN,NaN,Apartment,1381.0,1692.0,1.55,3.21,Apartment,1711.0,2343.0,1.76,4.79,Apartment,2423.0,2963.0,2.50,6.06,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
4,None,NaN,NaN,NaN,NaN,Independent Floor,981.0,1118.0,93.01,1.06,Independent Floor,1235.0,1530.0,1.12,1.45,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,None,NaN,NaN,NaN,NaN,Apartment,964.0,NaN,1.35,NaN,Apartment,1127.0,NaN,1.55,NaN,Apartment,1562.0,NaN,2.15,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
242,Apartment,335.0,398.0,23.45,27.86,Apartment,500.0,625.0,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
243,None,NaN,NaN,NaN,NaN,Apartment,1450.0,NaN,80.00,NaN,Apartment,1991.0,NaN,1.00,NaN,Apartment,2639.0,4711.0,1.20,2.14,Apartment,4731.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
244,None,NaN,NaN,NaN,NaN,Apartment,1000.0,1079.0,NaN,NaN,Apartment,1225.0,1865.0,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN


In [172]:
df['PriceDetails'].iloc[213]

"{'4 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '4,495 - 4,645 sq.ft.', 'price-range': '₹ 9.1 - 9.4 Cr'}, '5 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '5,330 - 5,500 sq.ft.', 'price-range': '₹ 13 - 13.41 Cr'}, '6 BHK': {'building_type': 'Apartment', 'area_type': 'Super Built-up Area', 'area': '6,068 - 6,425 sq.ft.', 'price-range': '₹ 15 - 15.88 Cr'}}"

In [174]:
new_df['building_type_Land_BHK'].dtype

dtype('O')

In [176]:
cat_columns = [i for i in new_df.columns if new_df[f"{i}"].dtype == 'O']

In [178]:
cat_columns

['building_type_1_BHK',
 'building_type_2_BHK',
 'building_type_3_BHK',
 'building_type_4_BHK',
 'building_type_5_BHK',
 'building_type_6_BHK',
 'building_type_1_RK',
 'building_type_Land_BHK']

In [180]:
ohe_df = pd.get_dummies(new_df, columns = cat_columns, drop_first=True)

In [182]:
ohe_df.fillna(0, inplace=True)

In [184]:
ohe_df.shape

(246, 43)

In [186]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

ohe_df_scaled = pd.DataFrame(scaler.fit_transform(ohe_df), columns= ohe_df.columns, index=ohe_df.index)

ohe_df_scaled

,area_low_1_BHK,area_high_1_BHK,price_low_1_BHK,price_high_1_BHK,area_low_2_BHK,area_high_2_BHK,price_low_2_BHK,price_high_2_BHK,area_low_3_BHK,area_high_3_BHK,price_low_3_BHK,price_high_3_BHK,area_low_4_BHK,area_high_4_BHK,price_low_4_BHK,price_high_4_BHK,area_low_5_BHK,area_high_5_BHK,price_low_5_BHK,price_high_5_BHK,area_low_6_BHK,area_high_6_BHK,price_low_6_BHK,price_high_6_BHK,area_low_1_RK,area_high_1_RK,price_low_1_RK,price_high_1_RK,area_low_Land_BHK,area_high_Land_BHK,price_low_Land_BHK,price_high_Land_BHK,building_type_1_BHK_Service Apartment,building_type_2_BHK_Independent Floor,building_type_2_BHK_Service Apartment,building_type_3_BHK_Independent Floor,building_type_3_BHK_Service Apartment,building_type_3_BHK_Villa,building_type_4_BHK_Independent Floor,building_type_4_BHK_Villa,building_type_5_BHK_Independent Floor,building_type_5_BHK_Villa,building_type_6_BHK_Villa
0,-0.252236,-0.121568,-0.128400,-0.111858,1.223655,-0.604921,-0.388845,-0.174181,0.553850,0.734623,-0.210192,0.515061,0.602857,-0.695777,0.303242,0.242019,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,-0.447025,-0.357058,-0.212932,-0.236510,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
1,-0.252236,-0.121568,-0.128400,-0.111858,-0.893477,-0.604921,-0.455791,-0.293156,0.293146,0.829472,-0.212754,0.342903,0.382765,0.445996,0.261343,0.108677,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,-0.447025,-0.357058,-0.212932,-0.236510,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
2,-0.252236,-0.121568,-0.128400,-0.111858,-0.893477,-0.604921,-0.455791,-0.293156,0.500645,1.604068,-0.200970,4.253341,0.696647,1.228560,0.334667,3.124152,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,0.369586,1.841351,-0.079340,6.344901,-0.111111,-0.289310,-0.063888,2.683282,-0.063888,-0.171139,3.924283,-0.236208,-0.111111,-0.216353,-0.063888
3,-0.252236,-0.121568,-0.128400,-0.111858,1.240653,1.986044,-0.403908,-0.134026,0.405941,0.966212,-0.235297,0.883970,0.492186,0.571292,0.109455,0.483000,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,-0.447025,-0.357058,-0.212932,-0.236510,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
4,-0.252236,-0.121568,-0.128400,-0.111858,0.622513,1.107076,2.657514,-0.240608,-0.100570,0.323613,-0.268088,-0.142827,-1.022825,-0.695777,-0.545232,-0.490563,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,-0.447025,-0.357058,-0.212932,-0.236510,-0.111111,3.456497,-0.063888,2.683282,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,-0.252236,-0.121568,-0.128400,-0.111858,0.596242,-0.604921,-0.410602,-0.293156,-0.215493,-0.885706,-0.246057,-0.588593,-0.046165,-0.695777,0.017799,-0.490563,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,-0.447025,-0.357058,-0.212932,-0.236510,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,-0.236208,-0.111111,-0.216353,-0.063888
242,1.565583,0.792178,4.330157,9.991313,-0.120801,0.352144,-0.455791,-0.293156,-1.414732,-0.885706,-0.325471,-0.588593,-1.022825,-0.695777,-0.545232,-0.490563,-0.468952,-0.329171,-0.290991,-0.235915,-0.125582,-0.08283,-0.077649,-0.073387,-0.105145,-0.102528,-0.090521,-0.082725,-0.447025,-0.357058,-0.212932,-0.236510,-0.111111,-0.289310,-0.063888,-0.372678,-0.063888,-0.171139,-0.254824,

In [188]:
sim2 = cosine_similarity(ohe_df_scaled)

In [190]:
def recommend_properties_price_details(x:str):
    ind_price = li_property_names.index(x)
    price_sims = sim2[ind_price].tolist()
    numbered_sims = list(enumerate(price_sims))
    closer = sorted(numbered_sims, key=lambda x: x[1], reverse=True)[:6]
    indices = []
    for idx, _ in closer:
        indices.append(idx)
    return df[['PropertyName']].loc[indices]

In [192]:
recommend_properties_price_details('M3M Golf Hills')

,PropertyName
16,M3M Golf Hills
29,BPTP Terra
77,DLF Park Place
186,Paras Dews
0,Smartworld One DXP
169,ILD Greens


In [194]:
un = set()
def uni(s):
    for i in ast.literal_eval(s).keys():
        un.add(i)        

In [196]:
for i in df['LocationAdvantages']:
    uni(i)

In [198]:
pattern_Km = re.compile(r"(\d+\.?\d*)\s*?(?:km\b)", flags=re.IGNORECASE)
pattern_m = re.compile(r"(\d+\.?\d*)\s*?(?:meters?|m\b)", flags=re.IGNORECASE)

In [505]:
def LocationPre(s):
    dic = ast.literal_eval(s)
    store_keys = []
    for key1, val in dic.items():
        
        match = re.findall(pattern_m, val)
        
        if match:
            value = str(float(match[0]) / 1000)
        else:
            try:
                value = re.findall(pattern_Km, val)[0]
            except:
                value = '1'
        location_dic[key1].append(value)
        store_keys.append(key1)
    for k, v in location_dic.items():
        if k not in store_keys:
            location_dic[k].append(np.nan)
                

In [507]:
location_dic = {}

for i in un:
    location_dic[i] = []

In [509]:
for i in df['LocationAdvantages']:
    LocationPre(i)

In [511]:
location_df = pd.DataFrame(location_dic)

In [513]:
location_df.fillna(54, inplace=True)

In [515]:
location_df.head()

,Kundli Manesar Palwal Expressway,RBSM Public school,DPS Sector 103,The Signature Advanced Super Speciality,Cyber ​​Park,Eye Doctors at Krishna Netralaya,Drona Sports Village,Scottish International School,Holiday Inn Sector 90,DLC Cricket Ground,DoubleTree by Hilton Hotel,Unicosmos School,Swastik Hospital,Ocus Medley Mall,Garhi Budhera Road,"Dwarka Expy, Dhanwapur Village",Sector 55-56 Metro station,NH 352W,DPSG Palam Vihar Gurugram,Old Sohna Dhani Road,EPF Regional Office,Sun's Spa,Delhi Jaipur Expressway,Canara Bank - Nawada Fatehpur,Aarvy Healthcare Hospital,VIBGYOR High School,Patil Station,Artemis Hospital,Lingaya's Lalita Devi Institute,"Tulip Violet Society, Sector 69",Diamond Public School,Sai Sports Club cricket ground,Narayana e Techno School,SGT UNIVERSITY,Medicity,Golf course extension road,Pathways,Canara Bank New Palam Vihar,City Square,OMAXE Gurgaon Mall,Royal Institute Of Science,Gurgaon Old Railway Station,Sector 55 Metro Station,Imperial Heritage School,Punjab National Bank,Capital Cyberscape,Heritage School,Rapid Metro Station Sector 56,Dhanwapur Road,CBR Cricket Ground,Shalom Hills International School,Pawlywoof - Dog Park,Yonex Badminton Stadium,"Park Hospital, Palam Vihar",Marriott Courtyard,Edge towers tennis court,Khatu Shyam Mandir,Daultabad Village Park,Le Meridien Gurgaon,Huda City Centre,Ascendas OneHub Gurgaon,Vipul Trade Centre,"NH 48, Sector 78",Skyview Corporate Park,PVR Drive In Theater,M3m 65th Avenue Mall,Heritage Intl Xperiential School,AIPL Joy Street Mall,JMS Marine Square Mall,Sapphire 83,Sant Soordas Sihi Metro Station,Signature tower,Sheetla Mata Mandir,HSBC,Aarvy Healthcare Super Speciality,National Highway 48,Cloudnine Hospital Sector 47,iGrow Montessori Play School,Lovely Public School,Kangaroo Kids Preschool,Matrikiran School,Star Mall,Bhondsi Nature Park,Suncity School,Colonel's Central Academy,"CK Birla Hospital, Gurgaon",IG International Airport,National Tennis Academy Sector 98,Vardaan hospital and trauma centre,MKD Hospital,Kamal Hospital,Vardaan Hospital,The Millennium School,Yashroop Hospital,Splendor Trade Tower,Krishna Hospital,The Paras World School,Guls' Kitchen,Rx Pharmacy,AIPL Business Centre,Sanjeevani Hospital - Child Specialist,Suncity Market,Gyaananda School,"International Tech Park Gurgaon,",IMT Office Sohna,Mall Fifty One,Trident Hotel Gurgaon,South point Mall,Deerika HyperMart,Signature Hospital,Sector 10 Market,"Fitso Sector 48 Spuddy, Badminton",SCC Rooftop Drive-In,Vatika Town Square-INXT,Signature Super Speciality Hospital,Nehru Stadium,Rajesh Pilot Road,Qutub Plaza,Sohna Road,ICICI Bank,Medanta The Medicity,"Euro International School, sector- 51",Gurjar Samrat Jaipal Khatana Marg,Women Police Station,ESIC HOSPITAL,Marengo Asia Hospitals,Palam Vihar Halt,Yashlok Medical Centre,HP Petrol Pump,AIIMS,ISKCON Temple,Vatika Town Square,Sanjivani Hospital,Old Bengali Market,Sector 55-56 Rapid Metro,The Vivekananda School,Medanta-The Medicity,Golf Course Extension Rd,Dwarka Expressway Link Road,V-Square Sohna New Residential,Sector-21 Metro Dwarka,CANARA BANK,Spaze Business Park,MGF Megacity Mall,Genesis Hospital Sector 84,InfinityS Badminton Academy,Kings International School,Gurugram Railway Station,Vatika Town Square-INXT Mall,DLF Golf and Country Club,G D Goenka World School,Tau Devi Lal Sports Complex,Huda City Centre Metro Station,Elan Miracle Mall,Airia Mall Sector 68,Civil Hospital,Canara Bank ATM,Ibis Hotel,VATIKA BUSINESS PARK Sohna Rd,Basai Dhankot,Prakash Hospital,Aravali Adventure Hill,"The Oberoi, Gurgaon",Heritage Badminton Academy,V'Lante Mall,The Sylvan Trails School,"Badshahpur Sohna Rd Hwy, Rajoria Ngr",Ajit Stadium Dhanwapur,Stymerra Chowk,Indus valley Public School,Sohna road dhunela,FUEL NATION,Kidzee Sec-93,Southern Periphery Road,NH248A,"Euro International School, Sector- 109",Daultabad Stadium,Anand Multispeciality Hospital,Damdama Lake,Frescos,Imt Manesar,Shikshantar - Primary School,"NH 8, Sector 15 Part 2","Gurgaon - Delhi Expy, Sector 75A

In [516]:
for i in location_df.columns:
    location_df[i] = location_df[i].astype('float')

In [518]:
location_df.shape

(246, 1070)

In [521]:
sim3 = cosine_similarity(location_df)

In [523]:
def recommend_properties_locationAdvantage(x:str):
    ind_location = li_property_names.index(x)
    location_sims = sim3[ind_location].tolist()
    numbered_sims = list(enumerate(location_sims))
    closer = sorted(numbered_sims, key=lambda x: x[1], reverse=True)[:6]
    indices = []
    for idx, _ in closer:
        indices.append(idx)
    return df[['PropertyName']].loc[indices], closer

In [525]:
recommend_properties_facilities('BPTP Spacio')

,PropertyName
244,BPTP Spacio
96,DLF New Town Heights 2
130,Corona Optus
136,DLF The Pinnacle
137,Godrej Oasis
178,Landmark The Homes 81


In [527]:
recommend_properties_price_details('BPTP Spacio')

,PropertyName
244,BPTP Spacio
104,M3M Sierra 68
32,Godrej Nature Plus Serenity
245,SS The Coralwood
232,Unitech Fresco
51,Smart World Orchard


In [529]:
recommend_properties_locationAdvantage('BPTP Spacio')

(                            PropertyName
 244                          BPTP Spacio
 214                BPTP Park Generations
 198                    Conscient Habitat
 23                       Godrej Meridien
 203            Signature Global Solera 2
 212  Optimal ultra luxury builder floors,
 [(244, 1.0000000000000016),
  (214, 0.9991057505742902),
  (198, 0.9964494081677069),
  (23, 0.9960859298309812),
  (203, 0.9959936719815229),
  (212, 0.9955358882953762)])

In [573]:
final_sim = 1*sim1 + 2*sim2 + 10*sim3

In [575]:
def recommend_properties_with_finalSimValue(x:str):
    ind = li_property_names.index(x)
    sims = final_sim[ind].tolist()
    numbered_sims = list(enumerate(sims))
    closer = sorted(numbered_sims, key=lambda x: x[1], reverse=True)[1:6]
    indices = []
    for idx, _ in closer:
        indices.append(idx)
    return df[['PropertyName']].loc[indices]

In [590]:
pd.DataFrame(final_sim).to_csv('../datasets/final_sim.csv', index=False)

In [580]:
recommend_properties_with_finalSimValue('DLF The Camellias')

,PropertyName
213,Salcon The Verandas
56,DLF The Magnolias
57,DLF The Aralias
117,Pioneer Urban Presidia
20,Tulip Monsella


In [537]:
location_df.set_index(df.PropertyName, inplace=True)
location_df.reset_index(inplace=True)

In [539]:
location_df.to_csv("../datasets/location_df.csv", index=False)

In [541]:
df.to_excel("../datasets/real_estate_data.xlsx", index=False)

In [596]:
df[df.PropertyName == "Mahindra Luminare"]['LocationAdvantages'].values

array(["{'Grand Hyatt Gurgaon': '1.7 Km', 'Duke Horse Riding Club': '2.1 Km', 'PVR Drive In Cinema': '2.2 Km', 'W Pratiksha Hospital': '2.6 Km', 'Sector 55-56 Metro Station': '2.7 Km', 'Metro World Mall': '3.7 Km', 'Unicosmos School': '3.9 Km', 'Sushant University': '4.4 Km', 'Faridabad Gurgaon Road': '4.9 Km', 'Sohna Road': '6.5 Km', 'Bestech Business Tower': '8.3 Km', 'Appu Ghar': '9.2 Km', 'SkyJumper Trampoline Park': '10 Km', 'Indira Gandhi International Airport': '20.6 Km', 'Axis Bank': '400 m'}"],
      dtype=object)

In [598]:
def get_hash(df):
        return hashlib.md5(pd.util.hash_pandas_object(df, index=True).values).hexdigest()

In [602]:
get_hash(location_df)

'484031023ff9f875aee764814e8f3b0a'